In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

from fastai.vision.all import *

In [ ]:
path = Path('../input/h-and-m-personalized-fashion-recommendations')
ar_df = pd.read_csv(path/'articles.csv')

In [ ]:
ar_df.info()

For at dette skal fungere må vi legge til tilhørende bilde for hver artikkel. Dette kan vi gjøre ved å lagre mappestien til bildet i en ny kollonne for tilhørende produkt

In [ ]:
ar_df['img_path'] = ""

In [ ]:
BASE = '../input/h-and-m-personalized-fashion-recommendations/images/'

for i in ar_df.index:
    a = ar_df.at[i, 'article_id']
    ar_df.at[i,"img_path"] = BASE+'0'+str(a)[:2]+'/0'+str(a)+'.jpg'

In [ ]:
# Ser om det er lagt til korrekt
ar_df.iloc[3000]['img_path']

In [ ]:
os.path.exists('../input/h-and-m-personalized-fashion-recommendations/images/010/0108775015.jpg')

In [ ]:
os.path.exists('../input/h-and-m-personalized-fashion-recommendations/images/053/0539724003.jpg')

In [ ]:
ar_df.iloc[10305]['img_path']

Ser at det er noen produkter som ikke har tilhørende bilde. Disse kan fjernes fra datasettet. 

In [ ]:
dar_df = ar_df.copy()
c = 0
for i in ar_df.index:
    if not os.path.exists(ar_df.at[i,"img_path"]):
        c = c + 1 #Teller hvor mange produkter som blir fjernet
        dar_df = dar_df.drop(i)
        
print(c)

In [ ]:
dar_df.info()

In [ ]:
ar_df.product_type_name.value_counts()

In [ ]:
uh = dar_df['product_type_name'].value_counts() < 20
#uh.index.values
uh.values

In [ ]:
# Ser hva de ulike kategoriene som inneholder mindre en 20 produkter er og fjerner disse produktene fra datasettet.
for i in range(131):
    if uh.values[i] == True:
        print(uh.index.values[i], uh.values[i])
        dar_df = dar_df.drop(dar_df[dar_df['product_type_name'] == uh.index.values[i]].index)

In [ ]:
# Sjekker hvor mange ulike kategorier som gjennstår.
dar_df['product_type_name'].value_counts() < 20

Ferdig med å lage til datasettet og kan nå begjynne å lage modellen

In [ ]:
sz=224
item_tfms = Resize(sz)
batch_tfms = None
bs=64

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                   get_x=ColReader('img_path'), 
                   get_y=ColReader('product_type_name'),
                   splitter=RandomSplitter(valid_pct=0.2, seed=42), 
                   item_tfms=item_tfms, 
                   batch_tfms=batch_tfms)

In [ ]:
dls = dblock.dataloaders(dar_df, path=path, bs=bs)

In [ ]:
#dblock.show_batch()
#[Errno 2] No such file or directory: '../input/h-and-m-personalized-fashion-recommendations/images/053/0539724003.jpg'
# Ellers ikke mulig å gjøre dette fordi bildene starter med 0, som ikke er lov i Python.

In [ ]:
learn = vision_learner(dls, resnet34, metrics=accuracy)

In [ ]:
learn.fine_tune(1)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix(figsize=(30,30))

In [ ]:
interp.most_confused()